**Import Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

**Model Oluşturma**

In [9]:
#Keras ile model oluşturmak için Sequential ve Functional apileri kullanılır.
#Sıralı katmanlar için => sequential
#Dallanan bir model yapısı için => functional
#Pooling layer: Boyut azaltmak için kullanılan katmandır.(önemli özellikleri tutup gereksiz özellikleri azaltır.)
#Model katmanı söylenirken convolution ve dense katmanları sayılır.(pooling katmanı sayılmaz)
model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)), #input shape mnist için=28,28,1 cifar için=32,32,3
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax'),
])

**Model Derleme**

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2304)              0         
                                                                 
 dense_4 (Dense)             (None, 64)               

**Mnist Veriseti Üzerinde Eğitim**

In [4]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#görüntü pikselleri 255'e bölünerek 0-1 aralığına normalize edilir.
#işlem yükü azalır
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/5
1875/1875 [==============================] - 15s 5ms/step - loss: 0.1351 - accuracy: 0.9595 - val_loss: 0.0453 - val_accuracy: 0.9864
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0450 - accuracy: 0.9857 - val_loss: 0.0341 - val_accuracy: 0.9889
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0314 - accuracy: 0.9901 - val_loss: 0.0298 - val_accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0224 - accuracy: 0.9930 - val_loss: 0.0290 - val_accuracy: 0.9903
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0175 - accuracy: 0.9943 - val_loss: 0.0337 - val_accuracy: 0.9889


**Cifar10 Veriseti Üzerinde Eğitim**

In [11]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#görüntü pikselleri 255'e bölünerek 0-1 aralığına normalize edilir.
#işlem yükü azalır
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Epoch 1/5
1563/1563 [==============================] - 10s 5ms/step - loss: 1.4629 - accuracy: 0.4739 - val_loss: 1.2176 - val_accuracy: 0.5766
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1147 - accuracy: 0.6100 - val_loss: 1.0620 - val_accuracy: 0.6285
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9778 - accuracy: 0.6587 - val_loss: 0.9969 - val_accuracy: 0.6528
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8899 - accuracy: 0.6902 - val_loss: 0.9198 - val_accuracy: 0.6862
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8221 - accuracy: 0.7157 - val_loss: 0.9310 - val_accuracy: 0.6843


**VGG16 Model Fine Tune**

In [12]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=max,
    classes=10,
    classifier_activation = 'softmax'
    )

vgg16.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

**Transfer Learning ve Fine Tune**
*  Transfer öğrenmede, genellikle önceden eğitilmiş katmanların ağırlıklarını dondurur ve yalnızca daha yüksek katmanların ağırlıklarını ince ayar yaparsınız veya önceden eğitilmiş katmanların üzerine ek katmanlar ekleyip eğitirsiniz.
*  Fine Tune, bir önceden eğitilmiş modeli kullanmanın yanı sıra, modelin tüm parametrelerini yeni görev veya veri kümesine daha iyi uyacak şekilde güncellemenizi içeren özel bir transfer öğrenme türüdür.

**GoogleNet Model Fine Tune**

GoogleNet'in en dikkat çekici özelliği, paralel olarak çalışan ve farklı boyutlardaki konvolüsyonları birleştiren "Inception"
adlı bir mimariye dayanmasıdır. Bu mimari, hesaplama maliyetini azaltırken aynı anda daha fazla derinlik sağlamayı amaçlar.
Inception mimarisi, 1x1, 3x3 ve 5x5 boyutlarında konvolüsyonları ve ayrıca max pooling katmanlarını içerir.
Bu paralel yapılar, farklı özelliklerin çıkarılmasına ve daha etkili öğrenmeye olanak tanır.

In [15]:
googlenet = tf.keras.applications.InceptionV3(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(75,75,3),
    pooling=max,
    classes=10,
    classifier_activation = 'softmax'
    )

googlenet.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

**Resnet Model Fine Tune**

ResNet, geleneksel olarak derin sinir ağlarının eğitiminde karşılaşılan "derinleştikçe azalan performans" sorununu çözmek için
tasarlanmıştır. Bu sorun, ağın derinleşmesiyle birlikte eğitim hatasının artması ve aşırı öğrenmenin önlenememesiyle ortaya
çıkar. ResNet bu sorunu, her bir katmanın çıktısının girişine eklenen bir "rezidüel bağlantı" ile çözer.
-vanishing gradient problem-

In [16]:
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=max,
    classes=10,
    classifier_activation = 'softmax'
    )

resnet.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

**Densenet Model Fine Tune**

In [17]:
densenet = tf.keras.applications.DenseNet121(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=max,
    classes=10,
    classifier_activation = 'softmax'
    )

densenet.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])